# Download watershed boundaries

In [1]:
import os
import pathlib

import pandas as pd
import geopandas as gpd
import requests

working_dir = os.path.join(
    pathlib.Path.home(), 'earth-analytics', 'data', 'fire-management')
if not os.path.exists(working_dir):
    os.makedirs(working_dir)
    
os.chdir(working_dir)

In [2]:
wbd_18_url = (
    "https://prd-tnm.s3.amazonaws.com/StagedProducts/"
    "Hydrography/WBD/HU2/Shape/WBD_18_HU2_Shape.zip")
wbd_18_dir = 'water-boundary-dataset-hu18'
wbd_18_path = os.path.join(wbd_18_dir, wbd_18_dir + '.zip')

# Cache WBD file
if not os.path.exists(wbd_18_dir):
    os.makedirs(wbd_18_dir)
    
    # Download full WBD 18 as zipfile
    response = requests.get(wbd_18_url)
    

    with open(wbd_18_path, 'wb') as wbd_18_file:
        wbd_18_file.write(response.content)

In [3]:
%%bash
unzip water-boundary-dataset-hu18/water-boundary-dataset-hu18.zip

Archive:  water-boundary-dataset-hu18/water-boundary-dataset-hu18.zip


replace Shape/ExternalCrosswalk.dbf? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


CalledProcessError: Command 'b'unzip water-boundary-dataset-hu18/water-boundary-dataset-hu18.zip\n'' returned non-zero exit status 1.

In [4]:
wbd_hu12_uri = 'zip://{zip_path}!{subpath}'.format(
    zip_path=wbd_18_path,
    subpath=os.path.join('Shape', 'WBDHU12.shp'))

wbd_hu12_gdf = gpd.read_file(wbd_hu12_uri)

In [8]:
watershed_names = [
    'Illilouette',
    'Strawberry Creek'
]

watershed_gdfs = []
for watershed_name in watershed_names:
    watershed_gdf = wbd_hu12_gdf[wbd_hu12_gdf.name.str.contains(watershed_name)]
    watershed_gdfs.append(watershed_gdf)
    
wbd_gdf = pd.concat(watershed_gdfs)
wbd_gdf.total_bounds

array([-119.59436018,   33.67830762, -116.67180579,   37.72574759])

In [9]:
class BoundingBox:
    
    def __init__(self, gpd_bounds):
        self.gpd_bounds = gpd_bounds
        self.llx, self.lly, self.urx, self.ury = gpd_bounds
        
    @property
    def ll_lon(self):
        return self.llx

yosemite_bbox = BoundingBox(wbd_gdf.total_bounds)
yosemite_bbox.ll_lon

-119.59436017688938

In [10]:
class EarthExplorerDownloader:
    
    base_url = "https://m2m.cr.usgs.gov/api/api/json/stable/{}"
    username_path = os.path.join(pathlib.Path.home(), '.ee_test_username')
    password_path = os.path.join(pathlib.Path.home(), '.ee_test_pass')
    
    def __init__(self, dataset_name, bbox, start, end, label):
        self.dataset_name, self.label = dataset_name, label
        self.bbox, self.start, self.end = bbox, start, end
        
        self.login()
        
    def post(self, endpoint, params):
        # Make POST request
        response = requests.post(
            self.base_url.format(endpoint), 
            json.dumps(params))
        response.raise_for_status()
        return response.json()['data']
        
    def login(self):
        if os.path.exists(self.username_path):
            with open(self.username_path, 'r') as username_file:
                username = username_file.read()
        else:
            username = input('Username: ')
            with open(self.username_path, 'w') as username_file:
                username_file.write(username)
                
        if os.path.exists(self.password_path):
            with open(self.password_path, 'r') as password_file:
                password = password_file.read()
        else:
            password = input('Password: ')
            with open(self.password_path, 'w') as password_file:
                password_file.write(password)
        params = {'username' : username, 'password' : password}
        self.api_key = self.post('login', params)

    @property
    def dataset_alias(self):
        dataset_alias = 'stuff'
        return dataset_alias
    
    def find_scenes(self):
        pass

    def find_available_products(self):
        pass
    
    def request_download(self):
        pass
    
    def check_download_status(self):
        pass
    
    def download(self):
        pass
    
    def logout(self):
        pass
    
naip_downloader = EarthExplorerDownloader(
    dataset_name='naip',
    bbox=yosemite_bbox,
    start='2003-01-01', end='2023-01-01',
    label='yosemite-naip-03-23')

In [11]:
naip_downloader.api_key

'eyJjaWQiOjI2Mjg2OTkyLCJzIjoiMTY3NTkwMjE0MiIsInIiOjkzOSwicCI6WyJ1c2VyIiwiZG93bmxvYWQiLCJvcmRlciJdfQ=='